In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
original_train = pd.read_csv("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/consolidated_training_data/train_final_multiple_sources.csv")
original_train.head()

,Unnamed: 0,text,label,data_origin
0,0,We analyze the relationship between climate ch...,1,GPT-generated data (old)
1,1,Using a dataset from the World Trade Organizat...,1,GPT-generated data (old)
2,2,This study examines the impact of school-based...,1,GPT-generated data (old)
3,3,We explore the impact of electricity access on...,1,GPT-generated data (old)
4,4,To investigate the relationship between social...,1,GPT-generated data (old)


In [3]:
train_eval_fb = pd.read_json("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/guardrails/negative/train_eval_feedback_negative.json")
train_eval_fb.rename(columns={0:"text"}, inplace=True)
train_eval_fb['label'] = 0
train_eval_fb['data_origin'] = "GPT 3.5: first iteration feedback - negative samples"

email_url = pd.read_json("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/guardrails/negative/emails_and_urls.json")
email_url.rename(columns={0:"text"}, inplace=True)
email_url['label'] = 0
email_url['data_origin'] = "GPT 3.5: email and URL guardrail - negative samples"

paper_names = pd.read_json("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/guardrails/negative/paper_names.json")
paper_names.rename(columns={0:"text"}, inplace=True)
paper_names['label'] = 0
paper_names['data_origin'] = "GPT 3.5: paper names and research context guardrail"

train_eval_fb_positive = pd.read_json("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/guardrails/positive/train_eval_feedback.json")
train_eval_fb_positive.rename(columns={0:"text"}, inplace=True)
train_eval_fb_positive['label'] = 1
train_eval_fb_positive['data_origin'] = "GPT 3.5: first iteration feedback - positive samples"

email_url_positive = pd.read_json("/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/guardrails/positive/emails_and_urls.json")
email_url_positive.rename(columns={0:"text"}, inplace=True)
email_url_positive['label'] = 1
email_url_positive['data_origin'] = "GPT 3.5: email and URL guardrail - positive samples"

print(train_eval_fb.shape)
print(email_url.shape)
print(paper_names.shape)
print(train_eval_fb_positive.shape)
print(email_url_positive.shape)


(407, 3)
(10000, 3)
(1391, 3)
(144, 3)
(455, 3)


In [19]:
original_train['label'].value_counts()
original_train['data_origin'].value_counts()

original_train_real = original_train.query('data_origin.str.startswith("Coleridge") == True')
original_train_synthetic = original_train.query('data_origin.str.startswith("Coleridge") == False')
original_train_synthetic.shape

(8353, 4)

In [40]:
paper_names.head()

,text,label,data_origin
0,This study investigates the impact of market v...,0,GPT 3.5: paper names and research context guar...
1,The paper by Johnson et al. explores the effec...,0,GPT 3.5: paper names and research context guar...
2,"In their research, Brown et al. analyze the re...",0,GPT 3.5: paper names and research context guar...
3,The study by Martinez et al. examines the effe...,0,GPT 3.5: paper names and research context guar...
4,Jones et al. discuss the role of social media ...,0,GPT 3.5: paper names and research context guar...


In [60]:
paper_names.shape

(1391, 3)

In [69]:
# Creating training data

train_data_dfs = [original_train_synthetic, 
                  train_eval_fb_positive, 
                  email_url_positive.head(455), 
                  train_eval_fb, 
                  email_url.head(2000),
                  paper_names.head(1391)
                  ]

train_data = pd.concat(train_data_dfs, ignore_index=True)

# Create val data

# Positive and negative val data
pos = original_train_real.query('label==1')
neg = original_train_real.query('label==0').head(4467)
papers = paper_names.tail(0)
email_url_pos = email_url_positive.tail(0)

val_data = pd.concat([pos,neg, papers, email_url_pos], ignore_index=True)


save_path = "/Users/jamesliounis/Documents/Projects/World Bank/NLP Project/data-use/data/training/consolidated_training_data/"
train_data.to_excel(save_path + "train_data_v2.xlsx")
val_data.to_excel(save_path + "val_data_v2.xlsx")


In [29]:
train_data['data_origin'].value_counts(

Generated with GPT-3.5-Turbo                            3666
Generated with Llama-70B - Temperature=0.5              2108
GPT 3.5: email and URL guardrail - negative samples     2000
GPT-generated data (old)                                1986
GPT 3.5: paper names and research context guardrail     1391
GPT 3.5: email and URL guardrail - positive samples      455
GPT 3.5: first iteration feedback - negative samples     407
Mixtral-7B - temperature=0                               254
Mixtral-7B - temperature=0.5                             254
GPT 3.5: first iteration feedback - positive samples     144
Generated with GPT-4                                      85
Name: data_origin, dtype: int64

In [37]:
train_data.query("data_origin =='GPT 3.5: paper names and research context guardrail'")['text'].to_list()[1]

'The paper by Johnson et al. explores the effects of climate change on agricultural production.'

# Building Training and Validation Data

In [70]:
val_data['label'].value_counts()

0    4467
1     254
Name: label, dtype: int64

In [71]:
train_data['label'].value_counts()

1    8952
0    3798
Name: label, dtype: int64

In [66]:
original_train_real.query('label == 0').shape

(6663, 4)

In [67]:
train_data.shape

(12750, 4)

In [68]:
val_data.shape

(2580, 4)